In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# import yfinance as yf
from scipy.signal import argrelextrema
# from collections import deque
# from matplotlib.lines import Line2D
import pandas_ta as ta
import ccxt
from datetime import datetime, timedelta
# import plotly.graph_objects as go
# import inspect 
import os
# import ccxt_data_fetch
from IPython.display import display, HTML
from tqdm import tqdm


# pd.set_option('display.max_rows', None)
pd.reset_option("display.max_rows")

In [14]:
def calc():
    # data=ccxt_data() # using it will reset all the params to default parms
    # symbol='ETH/USDT'

    global order
    global peaks_df
    global troughs_df

    if len(data) > 14:
    

        data['ta_rsi'] = ta.rsi(data['Close'], length = 14)

        coin_peak = argrelextrema(data['Close'].values, np.greater, order=order)[0]

        coin_peaks_df = data.iloc[coin_peak]['Close']
  

        ta_peak = argrelextrema(data['ta_rsi'].values, np.greater, order=order)[0]
        ta_peaks_df = data.iloc[ta_peak]['ta_rsi']
 

        coin_trough = argrelextrema(data['Close'].values, np.less, order=order)[0]
        coin_trough_df = data.iloc[coin_trough]['Close']

        ta_trough = argrelextrema(data['ta_rsi'].values, np.less, order=order)[0]
        ta_trough_df = data.iloc[ta_trough]['ta_rsi']

        peaks = [coin_peaks_df, ta_peaks_df]
        peaks_df = pd.concat(peaks, axis=1)
        peaks_df = peaks_df[(peaks_df.Close.notna()) & (peaks_df.ta_rsi.notna())]

        troughs = [coin_trough_df, ta_trough_df]
        troughs_df = pd.concat(troughs, axis=1)
        troughs_df = troughs_df[(troughs_df.Close.notna()) & (troughs_df.ta_rsi.notna())]

        if peaks_df.empty:
            # tof=False
            # print('empty peaks', symbol)
            return False

        elif troughs_df.empty:
            # print('empty troughs', symbol)
            return False
        
        else:
            return peaks_df, troughs_df, True

    else:
        # tof=False
        print('data length less than 14', symbol)
        return False




In [15]:

# peaks_df, troughs_df = calc()

def signal_dates(n=9):
    global sell
    global buy
    sell = []
    buy = []
    dates_of_peaks = []
    dates_of_troughs = []

    if calc():

        for i in range(1,n):

            diff = peaks_df.diff(i)
            # diff.dropna(inplace=True) # do not use it
            sig = diff[~((diff > 0).all(1)) & ~((diff < 0).all(1))]
            dates_of_peaks.append(sig.index.values)

    
            diff = troughs_df.diff(i)
            # diff.dropna(inplace=True) # do not use it
            sig = diff[~((diff > 0).all(1)) & ~((diff < 0).all(1))]
            dates_of_troughs.append(sig.index.values)

        for list in dates_of_peaks:
            for i in list:
                if str(i) not in sell:
                    # np.datetime to str
                    # maybe pd.to_datetime(i) works better
                    sell.append(str(i))

        for list in dates_of_troughs:
            for i in list:
                if str(i) not in buy:
                    buy.append(str(i))
                    

        sell.sort()
        buy.sort()
        # print(dates_of_peaks)
        # problem : buy ans sell should be list of datetime***********************

        # sell=pd.to_datetime(sell)
        # buy=pd.to_datetime(buy)


        if not buy or not sell:
            print('empty buy or sell')
            return False
        else:
            return sell, buy, True

    
            
    else:
        # tof=False
        # print(' calc() is false ', symbol)
        return False



In [16]:
def resulta():
    # rows =[] do not put it inside a loop 
    global table
  

    if signal_dates():

        sell_date = min((dt for dt in sell if dt > buy[-1]),default=buy[-1])
        # sell_date = sell_date + np.timedelta64(8, 'h')

        if sell_date > buy[-1]:
            trade_on = False
        else:
            trade_on = True


        selldate = sell_date # its buy[-1] bcase of default arg in min

        rows.append([symbol, buy[-1], selldate, trade_on]) 

        table = pd.DataFrame(rows, columns=["symbols", "Buy","Sell", "trade_status"])

        table['link'] = table['symbols'].apply(lambda x: f'<a href="https://www.tradingview.com/chart/UOC7kIDx/?symbol=BINANCE%3A{x.replace("/","")}">{x}</a>')


    else:
        # empty df 
        # but sometimes not empy like mbl and xec and rgt, tested it on 20/02/2022
        # i think maybe no signal found 
        rows.append([symbol, 0, 0, 0])
        table = pd.DataFrame(rows, columns=["A", "Buy","Sell", "trade_status"])
        
        # print('signal_dates is false ', symbol)
        bad_pairs.append(symbol)
        

    return table
    




In [17]:

n=9
order=5
rows = []
bad_pairs=[]
timeframe='4h'
outdir=f"{os.getcwd()}/data/{timeframe}"


# from ttm_squeeze.ipynb

# use this optin for candidates from another scipt

# %store -r list_of_candidate
# list_of_candidate
# for filename in tqdm(list_of_candidate):
#     symbol = filename
#     data = pd.read_csv(f"{outdir}/{filename}_{timeframe}.csv", index_col='Time')
#     calc()
#     signal_dates(n)
#     resulta()

# or use this to get the result of all coins
for filename in tqdm(os.listdir(outdir)):
    symbol = filename.replace(f"_{timeframe}.csv","")
    data = pd.read_csv(f"{outdir}/{filename}", index_col='Time')

    # data = data[:-2]
    
    # sppeed the process by selecting only the last 20 candles if the data
    # but it wont work; maybe fix it later
    # data = data[-50:]

    calc()
    signal_dates(n)
    resulta()

# pairs with no buy or sell signal
print(f"pairs with no signal : \n{bad_pairs}")

# display all coins
# display(HTML(table.to_html(escape=False)))

# display only the coins that no sell has been found yet
display(HTML(table[table.trade_status == True].to_html(escape=False)))



100%|██████████| 191/191 [00:28<00:00,  6.65it/s]

pairs with no signal : 
[]


,symbols,Buy,Sell,trade_status,link
0,1INCHUSDT,2022-02-24 00:00:00,2022-02-24 00:00:00,True,1INCHUSDT
3,ACHUSDT,2022-02-24 00:00:00,2022-02-24 00:00:00,True,ACHUSDT
4,ACMUSDT,2022-02-24 00:00:00,2022-02-24 00:00:00,True,ACMUSDT
10,ALCXUSDT,2022-02-24 00:00:00,2022-02-24 00:00:00,True,ALCXUSDT
13,ALPACAUSDT,2022-02-24 00:00:00,2022-02-24 00:00:00,True,ALPACAUSDT
20,API3USDT,2022-02-22 00:00:00,2022-02-22 00:00:00,True,API3USDT
22,ARPAUSDT,2022-02-24 00:00:00,2022-02-24 00:00:00,True,ARPAUSDT
25,ATAUSDT,2022-02-24 00:00:00,2022-02-24 00:00:00,True,ATAUSDT
29,AUTOUSDT,2022-02-24 00:00:00,2022-02-24 00:00:00,True,AUTOUSDT
30,AVAUSDT,2022-02-10 12:00:00,2022-02-10 12:00:00,True,AVAUSDT


In [18]:
# list_of_candidate